In [37]:
pip install pandas numpy matplotlib seaborn plotly

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Step 1 & 2: Load and Prepare Datasets (using the confirmed file and column names) ---

# Load Bitcoin Market Sentiment Dataset
try:
    sentiment_df = pd.read_csv('fear_greed_index.csv')
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
    sentiment_df.set_index('date', inplace=True)
    sentiment_df.sort_index(inplace=True)
    print("Sentiment Data prepared successfully.")
except Exception as e:
    print(f"Error preparing sentiment_df: {e}")
    sentiment_df = None

# Load Historical Trader Data
try:
    trader_data_df = pd.read_csv('historical_data.csv')
    # Convert the 'Timestamp' column (Unix timestamp in milliseconds) to datetime
    trader_data_df['Timestamp'] = pd.to_datetime(trader_data_df['Timestamp'], unit='ms')
    trader_data_df.set_index('Timestamp', inplace=True)
    trader_data_df.sort_index(inplace=True)
    print("Trader Data prepared successfully.")
except Exception as e:
    print(f"Error preparing trader_data_df: {e}")
    trader_data_df = None

# --- Step 3: Exploratory Data Analysis (EDA) - Basic Checks ---
if trader_data_df is not None:
    print("\nTrader Data Coin Value Counts:")
    print(trader_data_df['Coin'].value_counts())
    print("\nTrader Data Side Value Counts:")
    print(trader_data_df['Side'].value_counts())

    # Distribution of Closed PnL
    plt.figure(figsize=(10, 6))
    sns.histplot(trader_data_df['Closed PnL'].dropna(), bins=50, kde=True)
    plt.title('Distribution of Closed PnL')
    plt.xlabel('Closed PnL')
    plt.ylabel('Frequency')
    plt.savefig('closed_pnl_distribution.png')
    plt.close() # Close plot to free memory
    print("\n'closed_pnl_distribution.png' saved.")
else:
    print("trader_data_df is not available for EDA.")


# --- Step 4: Data Alignment and Merging ---
merged_df = None # Initialize merged_df outside the if block

if trader_data_df is not None and sentiment_df is not None:
    print("\nAttempting Data Alignment and Merging...")

    # Aggregate Trader Data Daily:
    # Using corrected column names ('Closed PnL', 'Account')
    # 'leverage' column is confirmed as not existing, so it's excluded.
    daily_trader_performance = trader_data_df.resample('D').agg(
        total_closedPnL=('Closed PnL', 'sum'),
        trade_count=('Account', 'size') # Count of trades per day based on 'Account'
    ).dropna()
    print("\nDaily Trader Performance Aggregation:")
    print(daily_trader_performance.head())

    # Ensure index names are consistent for merging
    sentiment_df.index.name = 'date'
    daily_trader_performance.index.name = 'date'

    # Ensure both dataframes cover overlapping date ranges for meaningful merge
    common_start = max(sentiment_df.index.min(), daily_trader_performance.index.min())
    common_end = min(sentiment_df.index.max(), daily_trader_performance.index.max())

    sentiment_df_filtered = sentiment_df[(sentiment_df.index >= common_start) & (sentiment_df.index <= common_end)]
    daily_trader_performance_filtered = daily_trader_performance[(daily_trader_performance.index >= common_start) & (daily_trader_performance.index <= common_end)]

    merged_df = pd.merge(daily_trader_performance_filtered, sentiment_df_filtered, on='date', how='inner')
    print("\nMerged Dataset:")
    print(merged_df.head())
    print(merged_df.info())
    print(f"\nMerged dataset covers {len(merged_df)} days from {merged_df.index.min().strftime('%Y-%m-%d')} to {merged_df.index.max().strftime('%Y-%m-%d')}")
else:
    print("\nCannot proceed with data alignment and merging. Ensure both sentiment_df and trader_data_df are loaded and processed.")

# --- Step 5: Explore Relationships and Uncover Patterns ---
if merged_df is not None:
    print("\nExploring Relationships and Uncovering Patterns (Step 5)...")

    # Trader Performance by Sentiment:
    print("\nTrader Performance by Sentiment Classification:")
    # Using 'classification' (lowercase) consistently. Removed average_leverage.
    sentiment_performance = merged_df.groupby('classification').agg(
        avg_daily_pnl=('total_closedPnL', 'mean'),
        median_daily_pnl=('total_closedPnL', 'median'),
        total_days=('classification', 'size')
    ).sort_values(by='avg_daily_pnl', ascending=False) # Sort for better readability
    print(sentiment_performance)

    # Visualize PnL distribution by sentiment
    plt.figure(figsize=(12, 7))
    sns.boxplot(x='classification', y='total_closedPnL', data=merged_df,
                order=['Extreme Greed', 'Greed', 'Neutral', 'Fear', 'Extreme Fear']) # Order categories
    plt.title('Daily Total Closed PnL by Market Sentiment')
    plt.xlabel('Sentiment Classification')
    plt.ylabel('Total Closed PnL')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('pnl_by_sentiment_boxplot.png')
    plt.close()
    print("\n'pnl_by_sentiment_boxplot.png' saved.")

    # Correlation Analysis (Example):
    # Mapping sentiment to numerical score (adjust mapping as per your interpretation)
    sentiment_mapping = {'Extreme Fear': -2, 'Fear': -1, 'Neutral': 0, 'Greed': 1, 'Extreme Greed': 2}
    merged_df['Sentiment_Score'] = merged_df['classification'].map(sentiment_mapping)

    if 'Sentiment_Score' in merged_df.columns:
        # Removed average_leverage from correlation matrix as it's not available
        correlation_matrix = merged_df[['total_closedPnL', 'trade_count', 'Sentiment_Score']].corr()
        print("\nCorrelation Matrix:")
        print(correlation_matrix)

        plt.figure(figsize=(8, 6))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Correlation Matrix of Performance Metrics and Sentiment')
        plt.savefig('correlation_heatmap.png')
        plt.close()
        print("\n'correlation_heatmap.png' saved.")

    # Additional Analysis (Example: PnL over time with sentiment overlay)
    plt.figure(figsize=(15, 7))
    plt.plot(merged_df.index, merged_df['total_closedPnL'], label='Daily Total Closed PnL', color='blue', alpha=0.7)

    # Overlay sentiment (e.g., color dots based on sentiment score)
    sentiment_colors = {
        'Extreme Greed': 'darkgreen',
        'Greed': 'limegreen',
        'Neutral': 'orange',
        'Fear': 'red',
        'Extreme Fear': 'darkred'
    }
    for sentiment, color in sentiment_colors.items():
        subset = merged_df[merged_df['classification'] == sentiment]
        plt.scatter(subset.index, subset['total_closedPnL'], color=color, label=f'Sentiment: {sentiment}', s=50, alpha=0.6, edgecolors='w', linewidth=0.5)

    plt.title('Daily Total Closed PnL with Market Sentiment Overlay')
    plt.xlabel('Date')
    plt.ylabel('Total Closed PnL')
    plt.legend(title='Sentiment')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('pnl_with_sentiment_overlay.png')
    plt.close()
    print("\n'pnl_with_sentiment_overlay.png' saved.")

else:
    print("\nCannot proceed with relationship exploration. Merged dataset not available.")

Sentiment Data prepared successfully.
Trader Data prepared successfully.

Trader Data Coin Value Counts:
Coin
HYPE    68005
@107    29992
BTC     26064
ETH     11158
SOL     10691
        ...  
@25         1
@86         1
@68         1
@45         1
@135        1
Name: count, Length: 246, dtype: int64

Trader Data Side Value Counts:
Side
SELL    108528
BUY     102696
Name: count, dtype: int64

'closed_pnl_distribution.png' saved.

Attempting Data Alignment and Merging...

Daily Trader Performance Aggregation:
            total_closedPnL  trade_count
Timestamp                               
2023-03-28              0.0            3
2023-03-29              0.0            0
2023-03-30              0.0            0
2023-03-31              0.0            0
2023-04-01              0.0            0

Merged Dataset:
            total_closedPnL  trade_count   timestamp  value classification
date                                                                      
2023-03-28              0.0    